In [49]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

Data loading

In [50]:
ratings = pd.read_csv("rate.csv",low_memory=False)
ratings_train = pd.read_csv("../Data_preprocess/Kaggle/ratios/edge_80.csv",low_memory=False)
ratings_test = pd.read_csv("rate_test.csv",low_memory=False)

In [51]:
datasets_train = ratings_train.dataset_id.unique()
model_train = ratings_train.model_id.unique()
datasets_test = ratings_test.dataset_id.unique()
model_test = ratings_test.model_id.unique()
models = pd.read_csv("models.csv",low_memory=False).model_id.unique()

Rating Matrix

In [52]:
data_model_train_matrix = pd.DataFrame(index=datasets_train,columns=models)
data_model_test_matrix = pd.DataFrame(index=datasets_test,columns=models)

In [53]:
for row in ratings_train.itertuples():
    data_model_train_matrix.loc[row[1]][row[2]] = row[3]
data_model_train_matrix = data_model_train_matrix.fillna(0)

In [54]:
for row in ratings_test.itertuples():
    data_model_test_matrix.loc[row[1]][row[2]] = row[3]
data_model_test_matrix = data_model_test_matrix.fillna(0)

Dataset Similarity Matrix

In [58]:
def similarity(user1, user2):
    # 获取两个用户共同评分的物品
    cols = data_model_train_matrix.columns[user1.values.nonzero()[0]]
    common_items = np.intersect1d(cols,data_model_train_matrix.columns[user2.values.nonzero()[0]]).tolist()
    # 如果没有共同评分的物品，则相似度为0
    if len(common_items) == 0:
        return 0
    # 计算两个用户共同评分物品的评分差的平方和
    diff = 0
    for common_item in common_items:
        diff += user1[common_item] - user2[common_item]
    square_diff = np.sum(diff**2)
    # 计算相似度
    sim = 1 / (1 + np.sqrt(square_diff))
    return sim

In [60]:
user_similarities = pd.DataFrame(index=datasets_train,columns=datasets_train)
for i in datasets_train:
    for j in datasets_train:
        user_similarities.loc[i][j] = similarity(data_model_train_matrix.loc[i], data_model_train_matrix.loc[j])
        user_similarities.loc[j][i] = user_similarities.loc[i][j]

In [62]:
user_similarities

,48,50,57,33,8,56,12,37,51,23,...,25,58,3,66,24,59,42,44,46,41
48,1.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
50,0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0.565563,0,0,0,0,0,0.5056
57,0,0,1.0,0,0,0,0,0,0,0,...,0,0,0,0,0.15517,0,0,0,0,0
33,0,0,0,1.0,0,0,0,0,0,0,...,0,0,0.419508,0,0,0,0,0,0,0
8,0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1.0,0,0,0,0
42,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1.0,0,0,0
44,0,0,0,0,0,0,0,0,0,0,...,0,0.891634,0,0,0,0,0,1.0,0,0
46,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.0,0


In [63]:
# dataset_similarity = cosine_similarity(data_model_train_matrix)
# model_similarity = cosine_similarity(data_model_train_matrix.T)
#
# dataset_similarity_csv = pd.DataFrame(dataset_similarity,index=datasets,columns=datasets)

Prediction

In [64]:
def predict(ratings, similarity):
        mean_user_rating = ratings.mean(axis=1)
        ratings_diff = ratings - np.array(mean_user_rating)[:,np.newaxis]
        pred = np.array(mean_user_rating)[:,np.newaxis] + np.dot(similarity,ratings_diff) / np.array([np.abs(similarity).sum(axis = 1)]).T
        return pred

In [65]:
user_prediction = predict(data_model_train_matrix,user_similarities)
model_prediction_csv = pd.DataFrame(user_prediction,index=datasets_train,columns=models).sort_index().sort_index(axis=1)
# model_prediction_csv.to_csv('model_prediction_csv',sep="\t")

In [66]:
model_prediction_csv

,72,73,74,75,76,77,78,79,80,81,...,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871
0,0.767445,0.792423,0.725187,0.900871,0.8595,0.877312,0.708945,0.757578,0.871463,0.739112,...,0.001378,0.001378,0.001378,0.001378,0.001378,0.001378,0.001378,0.001378,0.001378,0.001378
2,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,...,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937,0.000937
3,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,0.000148,...,0.077809,0.083958,0.080668,0.109437,0.121942,0.151469,0.056524,0.128739,0.205597,0.125543
4,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,...,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452,-0.000452
5,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,...,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688,-0.000688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,...,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279,-0.000279
68,0.138431,0.352461,0.333684,0.490662,0.276993,0.398343,0.059459,0.348159,0.287982,0.224719,...,-0.000795,-0.000795,-0.000795,-0.000795,-0.000795,-0.000795,-0.000795,-0.000795,-0.000795,-0.000795
69,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,...,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175,0.000175
70,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,...,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396,-0.000396


Validation

In [68]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [69]:
def rmse(prediction, ground_truth):
    prediction = prediction.to_numpy()[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction,ground_truth))

Metadata Based

In [74]:
meta_datasets = pd.read_csv("datasets_v.csv",low_memory=False)
datasets = meta_datasets.data_id.unique()
meta_datasets = meta_datasets.loc[:,('v1','v2','v3','v4','v5','v6','v7','v8')]
meta_datasets

,v1,v2,v3,v4,v5,v6,v7,v8
0,0.454312,0.241966,-0.178474,0.078718,-0.343706,0.150178,0.103396,-0.474233
1,0.527839,0.284245,-0.201757,0.116918,-0.394861,0.168884,0.137184,-0.536491
2,0.421102,0.234788,-0.172241,0.081514,-0.348054,0.138399,0.100450,-0.447096
3,0.358012,0.198786,-0.129784,0.068405,-0.275457,0.119678,0.093582,-0.378495
4,0.298305,0.167896,-0.120117,0.052906,-0.242469,0.107290,0.072812,-0.322821
...,...,...,...,...,...,...,...,...
67,0.446627,0.241174,-0.179276,0.092747,-0.355247,0.140657,0.110482,-0.472832
68,0.411207,0.244925,-0.156990,0.084651,-0.319750,0.147412,0.104277,-0.440762
69,0.495822,0.270633,-0.181879,0.100016,-0.387360,0.150677,0.108678,-0.507951
70,0.406016,0.214886,-0.155250,0.082475,-0.315015,0.126135,0.087693,-0.414519


In [75]:
array = meta_datasets.values.tolist()
array

[[0.454311557,
  0.241965768,
  -0.178474429,
  0.078717657,
  -0.343706069,
  0.150177872,
  0.103396425,
  -0.474233454],
 [0.527839024,
  0.284245372,
  -0.201756938,
  0.116918329,
  -0.394860626,
  0.16888419,
  0.137184327,
  -0.536490508],
 [0.421102265,
  0.234787548,
  -0.172240752,
  0.081513668,
  -0.348053636,
  0.138399188,
  0.100449697,
  -0.447095778],
 [0.358011712,
  0.198785837,
  -0.12978356,
  0.068405273,
  -0.275457302,
  0.119678099,
  0.09358213,
  -0.378495172],
 [0.298304723,
  0.167896053,
  -0.120116891,
  0.052905728,
  -0.242468698,
  0.1072904,
  0.072812415,
  -0.322821359],
 [0.348415455,
  0.188208281,
  -0.133945905,
  0.068413448,
  -0.268896279,
  0.101534269,
  0.093553875,
  -0.365702429],
 [0.459701074,
  0.243700217,
  -0.18333345,
  0.10144472,
  -0.376414413,
  0.156660285,
  0.118015787,
  -0.487984053],
 [0.498960363,
  0.288728784,
  -0.205670869,
  0.130357762,
  -0.373291447,
  0.151090336,
  0.125688079,
  -0.495753856],
 [0.405267065,


In [76]:
cosine_sim_1 = cosine_similarity([[0,1,-1,0],[1,1,-1,0]])
cosine_sim_2 = cosine_similarity([[0,1,-1,0],[1,-1,0,-1]])
cosine_sim_1

array([[1.        , 0.81649658],
       [0.81649658, 1.        ]])

In [77]:
meta_dataset_similarity = cosine_similarity(array)

meta_dataset_similarity = pd.DataFrame(meta_dataset_similarity)
meta_dataset_similarity

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,1.000000,0.999340,0.999456,0.999575,0.999625,0.999396,0.999243,0.997841,0.999250,0.999323,...,0.997632,0.999130,0.999007,0.999539,0.998801,0.999608,0.999271,0.999481,0.999642,0.948047
1,0.999340,1.000000,0.999077,0.999566,0.998910,0.999598,0.999372,0.999247,0.999423,0.999505,...,0.999322,0.999076,0.999649,0.999737,0.999076,0.999579,0.999309,0.999431,0.999504,0.948487
2,0.999456,0.999077,1.000000,0.999289,0.999741,0.999340,0.999700,0.998044,0.999652,0.999843,...,0.997227,0.999587,0.999390,0.999515,0.999457,0.999807,0.999147,0.999569,0.999624,0.944207
3,0.999575,0.999566,0.999289,1.000000,0.999517,0.999611,0.999329,0.998057,0.999114,0.999539,...,0.998123,0.999335,0.999438,0.999833,0.999392,0.999538,0.999702,0.999474,0.999356,0.943553
4,0.999625,0.998910,0.999741,0.999517,1.000000,0.999077,0.999478,0.997369,0.999099,0.999491,...,0.996876,0.999359,0.998899,0.999494,0.999082,0.999532,0.999482,0.999214,0.999278,0.943514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.999608,0.999579,0.999807,0.999538,0.999532,0.999761,0.999806,0.998538,0.999865,0.999877,...,0.998236,0.999621,0.999757,0.999848,0.999370,1.000000,0.999210,0.999676,0.999773,0.944395
68,0.999271,0.999309,0.999147,0.999702,0.999482,0.999029,0.999009,0.998374,0.998567,0.999303,...,0.997998,0.998751,0.998966,0.999500,0.999027,0.999210,1.000000,0.999157,0.999015,0.943228
69,0.999481,0.999431,0.999569,0.999474,0.999214,0.999483,0.999351,0.998503,0.999489,0.999736,...,0.997748,0.999103,0.999577,0.999528,0.999523,0.999676,0.999157,1.000000,0.999910,0.944969
70,0.999642,0.999504,0.999624,0.999356,0.999278,0.999441,0.999515,0.998538,0.999645,0.999690,...,0.997892,0.999110,0.999507,0.999552,0.999232,0.999773,0.999015,0.999910,1.000000,0.946840


In [78]:
i, j = 0, 0
while i < len(datasets):
    while j < len(datasets):
        meta_dataset_similarity[i][j] = None
        i += 1
        j += 1


In [79]:
meta_dataset_similarity

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
0,NaN,0.999340,0.999456,0.999575,0.999625,0.999396,0.999243,0.997841,0.999250,0.999323,...,0.997632,0.999130,0.999007,0.999539,0.998801,0.999608,0.999271,0.999481,0.999642,0.948047
1,0.999340,NaN,0.999077,0.999566,0.998910,0.999598,0.999372,0.999247,0.999423,0.999505,...,0.999322,0.999076,0.999649,0.999737,0.999076,0.999579,0.999309,0.999431,0.999504,0.948487
2,0.999456,0.999077,NaN,0.999289,0.999741,0.999340,0.999700,0.998044,0.999652,0.999843,...,0.997227,0.999587,0.999390,0.999515,0.999457,0.999807,0.999147,0.999569,0.999624,0.944207
3,0.999575,0.999566,0.999289,NaN,0.999517,0.999611,0.999329,0.998057,0.999114,0.999539,...,0.998123,0.999335,0.999438,0.999833,0.999392,0.999538,0.999702,0.999474,0.999356,0.943553
4,0.999625,0.998910,0.999741,0.999517,NaN,0.999077,0.999478,0.997369,0.999099,0.999491,...,0.996876,0.999359,0.998899,0.999494,0.999082,0.999532,0.999482,0.999214,0.999278,0.943514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.999608,0.999579,0.999807,0.999538,0.999532,0.999761,0.999806,0.998538,0.999865,0.999877,...,0.998236,0.999621,0.999757,0.999848,0.999370,NaN,0.999210,0.999676,0.999773,0.944395
68,0.999271,0.999309,0.999147,0.999702,0.999482,0.999029,0.999009,0.998374,0.998567,0.999303,...,0.997998,0.998751,0.998966,0.999500,0.999027,0.999210,NaN,0.999157,0.999015,0.943228
69,0.999481,0.999431,0.999569,0.999474,0.999214,0.999483,0.999351,0.998503,0.999489,0.999736,...,0.997748,0.999103,0.999577,0.999528,0.999523,0.999676,0.999157,NaN,0.999910,0.944969
70,0.999642,0.999504,0.999624,0.999356,0.999278,0.999441,0.999515,0.998538,0.999645,0.999690,...,0.997892,0.999110,0.999507,0.999552,0.999232,0.999773,0.999015,0.999910,NaN,0.946840


In [80]:
maxs = meta_dataset_similarity.max(axis=0)
maxs

0     0.999670
1     0.999773
2     0.999916
3     0.999833
4     0.999770
        ...   
67    0.999920
68    0.999702
69    0.999910
70    0.999910
71    0.955370
Length: 72, dtype: float64

In [81]:
def find_sim_index(index):
    row1 = meta_dataset_similarity.loc[index]
    row1_max_index = row1[row1 == row1.max()].index[0]
    return row1_max_index

In [ ]:
model_prediction_csv

In [84]:
for i in range(len(datasets)):
    if model_prediction_csv.loc[i].isnull().any():
        max_index = find_sim_index(i)
        print(model_prediction_csv.loc[i])
        model_prediction_csv.loc[i] = model_prediction_csv.loc[max_index]
        print(model_prediction_csv.loc[i])
    else:
        print("pass")

pass


KeyError: 1

In [ ]:
max_index = find_sim_index(1)
max_index

In [ ]:
model_prediction_csv.loc[67][225]

In [ ]:
data_model_test_numpy = np.array(data_model_test_matrix)

In [ ]:
# str(rmse(model_prediction_csv,data_model_test_numpy))

In [ ]:
# from sklearn.metrics import ndcg_score
# ndcg_score(model_prediction_csv,data_model_test_numpy)

In [ ]:
data_model_test_matrix = data_model_test_matrix[(data_model_test_matrix.T !=0).any()]
data_model_test_matrix = data_model_test_matrix.loc[[1,10,15,17,32,35,47,61,62,67]]
data_model_test_matrix

In [ ]:
index_list = data_model_test_matrix.index.to_list()
index_list

In [ ]:
model_ranking_groundtruth = pd.DataFrame(index=range(len(datasets)),columns=range(len(models)))
model_ranking_groundtruth

In [ ]:
for i in index_list:
    for j in range(len(models)):
        # index = index_list[i]
        row = data_model_test_matrix.loc[i]
        row = row.sort_values(ascending=False)
        index_row = row.index
        index_row = pd.DataFrame(index_row).T
        model_ranking_groundtruth.loc[i][j] = index_row.loc[0][j]
model_ranking_groundtruth

model_ranking_groundtruth = model_ranking_groundtruth[~model_ranking_groundtruth.isna().any(axis=1)]

In [ ]:
model_ranking_groundtruth

In [ ]:
row = model_prediction_csv.loc[1]
row = row.sort_values(ascending=False)
row = row.head(70).index
row = pd.DataFrame(row).T
# row.loc[0][1]
row

In [ ]:
model_ranking_prediction = pd.DataFrame(index=range(len(datasets)),columns=range(len(models)))
model_ranking_prediction

In [ ]:
for i in range(len(datasets)):
    for j in range(len(models)):
        row = model_prediction_csv.loc[i]
        row = row.sort_values(ascending=False)
        row = row.index
        row = pd.DataFrame(row).T
        model_ranking_prediction.loc[i][j] = row.loc[0][j]
model_ranking_prediction

In [ ]:
model_ranking_prediction.to_csv("model_ranking_prediction",sep='\t')

In [ ]:
model_ranking_prediction = model_ranking_prediction.loc[[1,10,15,17,32,35,47,61,62,67]]
model_ranking_prediction

In [ ]:
model_ranking_groundtruth

In [ ]:
model_ranking_prediction = model_ranking_prediction.values.tolist()

In [ ]:
data_model_test_matrix

In [ ]:
data_model_test_matrix.loc[1][1023]

In [ ]:
len(model_ranking_prediction[0])

In [ ]:
for i in range(len(model_ranking_prediction)):
    for j in range(len(model_ranking_prediction[i])):
        if data_model_test_matrix.loc[index_list[i]][model_ranking_prediction[i][j]]  == 0:
            model_ranking_prediction[i][j] = None
            print("删除了")
        # else:
        #     print("没事")
model_ranking_prediction

In [ ]:
# for i in range(len(model_ranking_prediction)):
#     for j in range(len(model_ranking_prediction[i])):
#         if model_ranking_prediction[i][j] not in model_ranking_groundtruth.values.tolist()[i]:
#             model_ranking_prediction[i][j] = None
#             print("删除了")
#         # else:
#         #     print("没事")
# model_ranking_prediction = model_ranking_prediction

In [ ]:
res = []
for i in range(len(model_ranking_prediction)):
    res.append(list(filter(None,model_ranking_prediction[i])))
res = pd.DataFrame(res,index=[1,10,15,17,32,35,47,61,62,67])

In [ ]:
res = res.fillna("None")

In [ ]:
res

In [ ]:
model_ranking_groundtruth

In [ ]:
model_ranking_prediction = pd.DataFrame(model_ranking_prediction)
model_ranking_prediction

In [ ]:
data_model_test_matrix.loc[32][889]
model_prediction_csv.loc[32][777]

In [ ]:
new_result = pd.DataFrame(columns={"dataset","model","predict_balance_accuracy","groundtruth_balance_accuracy"})
new_result

In [ ]:
data_model_test_matrix

In [ ]:
res.loc[:,:19]

In [ ]:
for i in range(10):
    dataset_id = index_list[i]
    for j in range(250):
        if res.loc[dataset_id][j] == "None":
            break
        model_id = res.loc[dataset_id][j]
        predict_balance_accuracy = model_prediction_csv.loc[dataset_id][model_id]
        groundtruth_balance_accuracy = data_model_test_matrix.loc[dataset_id][model_id]
        new_result = new_result.append([{'dataset':dataset_id,'model':model_id,'predict_balance_accuracy':predict_balance_accuracy,'groundtruth_balance_accuracy':groundtruth_balance_accuracy}],ignore_index=True)
#
# for row in top_20.itertuples():
#     new_result[row[2]][row[1]] = row[3]
# data_model_test_matrix = data_model_test_matrix.fillna(0)

In [ ]:
new_result

In [ ]:
new_result.to_csv("../Data_preprocess/Kaggle/CF_ratios_result/r80.csv")